In [2]:
import os
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-11"
os.environ["PATH"] = os.environ["JAVA_HOME"] + r"\bin;" + os.environ["PATH"]

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lr").getOrCreate()

In [3]:
training = spark.read.csv("test1.csv", header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [11]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["age", "Experience"], outputCol="Independent Features")

In [12]:
output = assembler.transform(training)

In [13]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [14]:
finalised_data = output.select("Independent Features", "Salary")

In [15]:
finalised_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [19]:
from pyspark.ml.regression import LinearRegression
#train_test_split
train_data, test_data = finalised_data.randomSplit([0.7, 0.3])
lr = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
lr_model = lr.fit(train_data)

In [21]:
lr_model.coefficients

DenseVector([-90.5483, 1608.7819])

In [22]:
lr_model.intercept

16079.136690647425